# Mount Drive

In [ ]:
#Allows dataset from drive to be utilized
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

## Suppress FutureWarnings
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_predict, GridSearchCV, cross_val_score, train_test_split, cross_validate, StratifiedKFold
from sklearn.metrics import confusion_matrix, make_scorer, recall_score, precision_score, f1_score, accuracy_score, roc_auc_score, auc, plot_roc_curve
from sklearn.preprocessing import MinMaxScaler 
from xgboost import XGBClassifier
from imblearn.metrics import geometric_mean_score
from imblearn.pipeline import Pipeline
from scipy.stats import mode
from sklearn.dummy import DummyClassifier
import statistics
from imblearn.under_sampling import ClusterCentroids
from imblearn.over_sampling import RandomOverSampler,SMOTE
import matplotlib.pyplot as plt
from imblearn.pipeline import Pipeline
from sklearn.model_selection import permutation_test_score
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier


# Import Dataset

In [ ]:
#Import DataFrame from .csv file
df = pd.read_csv(DATASET_LOCATION)

#Creating labels
x = df.drop("hgd_malignancy", axis=1); #Entire dataset
Y = df["hgd_malignancy"].copy();
feature_cols = x.columns

#Scale values from 0 to 1
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(x)
print(Y)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=12,test_size=.2,shuffle=True,stratify=Y)


# Import Strictly Texture Feature Dataset

In [ ]:
#Import DataFrame from .csv file
df = pd.read_csv(DATASET_LOCATION)

#Creating labels
x = df.drop("hgd_malignancy", axis=1); #Entire dataset
Y = df["hgd_malignancy"].copy();
feature_cols = x.columns

#Scale values from 0 to 1
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(x)
print(Y)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=12,test_size=.2,shuffle=True,stratify=Y)

# Import Non - Texture Feature Dataset

In [ ]:
#Import DataFrame from .csv file
df = pd.read_csv(DATASET_LOCATION)

#Creating labels
x = df.drop("hgd_malignancy", axis=1); #Entire dataset
Y = df["hgd_malignancy"].copy();
feature_cols = x.columns

#Scale values from 0 to 1
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(x)
print(Y)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=12,test_size=.2,shuffle=True,stratify=Y)

In [ ]:
import sys
print(sys.version)
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

# Hyperparameter Optimization

Best parameters optimized for G-mean (5-fold stratified CV): {'max_depth': 5, 'n_estimators': 9, 'scale_pos_weight': 2.46999999999999}



In [ ]:
# estimate scale_pos_weight value
# Here we set the estimate variable to the value of (minority class)/(majority class) 
# as a starting point for exploring different scale_pos_weight values
estimate = 54/26 
#54: Negative patients
#26: positive patients
print('Estimate: %f' % estimate)


In [ ]:
# Hyper-parameter Optimization

# Hyper-parameter Optimization
## Using hyper-parameter optimization, we found the best hyperparameters for
## our various models. 

## The specific hyperparameter values seen throughout the  notebook may not 
## necessarily be representative of exact hyperparameters used to achieve values
##  in manuscript
metric=make_scorer(geometric_mean_score)
weightlist= np.arange(1, 2, 0.02).tolist()
weightlist.append(estimate)
cv = StratifiedKFold(n_splits=5, shuffle=True)
model = XGBClassifier()

# Based on available compute time, set values for each hyperparameter in larger
# increments and becoming more granular on subsequent runs as we narrow down
# optimal parameter values
param_grid = [{'n_estimators': [5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40],
                  'max_depth': [3,4,5,6,7,8,9,10,11,12],
                  'scale_pos_weight': weightlist, # ratio to reverse of ratio with 0.05 in between
               }]; 

grid_search = GridSearchCV(model, param_grid, cv=cv, scoring=metric, )
grid_search.fit(X, Y)
best_model = grid_search.best_estimator_
print(grid_search.best_params_)

# Baseline Metrics from Various Models


## SVM

In [ ]:
Precisions = []
Recalls = []
F1s = []
G_means = []
accuracy = []
AUC = []
Specificities = []

for i in range(500):
  cv = StratifiedKFold(n_splits=5, shuffle=True)
  for train_fold_index, val_fold_index in cv.split(X,Y):
    X_train_fold_resample,y_train_fold_resample = X[train_fold_index], Y[train_fold_index]
    X_val_fold, y_val_fold = X[val_fold_index], Y[val_fold_index]
    model = svm.SVC()
    model.fit(X_train_fold_resample,y_train_fold_resample)
    pt = model.predict(X_val_fold)
    
    tn, fp, fn, tp = confusion_matrix(y_val_fold, pt).ravel()
    specificity = tn / (tn+fp)
    Specificities.append(specificity)
    Precisions.append(precision_score(y_val_fold,pt))
    Recalls.append(recall_score(y_val_fold,pt))
    F1s.append(f1_score(y_val_fold,pt))
    G_means.append(geometric_mean_score(y_val_fold,pt))
    accuracy.append(accuracy_score(y_val_fold,pt))
    AUC.append(roc_auc_score(y_val_fold,pt))

print('Precision- Mean:  %.3f Standard Deviation: %.3f' % (mean(Precisions), statistics.pstdev(Precisions)))
print('Sensitivity/Recall- Mean:  %.3f Standard Deviation: %.3f' % (mean(Recalls), statistics.pstdev(Recalls)))
print('Specificity - Mean:  %.3f Standard Deviation: %.3f' % (mean(Specificities), statistics.pstdev(Specificities)))
print('F1- Mean:  %.3f Standard Deviation: %.3f' % (mean(F1s), statistics.pstdev(F1s)))
print('G_mean- Mean:  %.3f Standard Deviation: %.3f' % (mean(G_means), statistics.pstdev(G_means)))
print('Accuracy- Mean:  %.3f Standard Deviation: %.3f' % (mean(accuracy), statistics.pstdev(accuracy)))
print('AUC Score- Mean:  %.3f Standard Deviation: %.3f' % (mean(AUC), statistics.pstdev(AUC)))

## Random Forest



In [ ]:
Precisions = []
Recalls = []
F1s = []
G_means = []
accuracy = []
AUC = []
Specificities = []

for i in range(500):
  cv = StratifiedKFold(n_splits=5, shuffle=True)
  for train_fold_index, val_fold_index in cv.split(X,Y):
    X_train_fold_resample, y_train_fold_resample = X[train_fold_index], Y[train_fold_index]
    X_val_fold, y_val_fold = X[val_fold_index], Y[val_fold_index]
    model = RandomForestClassifier(n_estimators=25,max_depth=20, class_weight='balanced')
    model.fit(X_train_fold_resample,y_train_fold_resample)
    pt = model.predict(X_val_fold)
    tn, fp, fn, tp = confusion_matrix(y_val_fold, pt).ravel()
    specificity = tn / (tn+fp)
    Specificities.append(specificity)
    Precisions.append(precision_score(y_val_fold,pt))
    Recalls.append(recall_score(y_val_fold,pt))
    F1s.append(f1_score(y_val_fold,pt))
    G_means.append(geometric_mean_score(y_val_fold,pt))
    accuracy.append(accuracy_score(y_val_fold,pt))
    AUC.append(roc_auc_score(y_val_fold,pt))

print('Precision- Mean:  %.3f Standard Deviation: %.3f' % (mean(Precisions), statistics.pstdev(Precisions)))
print('Sensitivity/Recall- Mean:  %.3f Standard Deviation: %.3f' % (mean(Recalls), statistics.pstdev(Recalls)))
print('Specificity - Mean:  %.3f Standard Deviation: %.3f' % (mean(Specificities), statistics.pstdev(Specificities)))
print('F1- Mean:  %.3f Standard Deviation: %.3f' % (mean(F1s), statistics.pstdev(F1s)))
print('G_mean- Mean:  %.3f Standard Deviation: %.3f' % (mean(G_means), statistics.pstdev(G_means)))
print('Accuracy- Mean:  %.3f Standard Deviation: %.3f' % (mean(accuracy), statistics.pstdev(accuracy)))
print('AUC Score- Mean:  %.3f Standard Deviation: %.3f' % (mean(AUC), statistics.pstdev(AUC)))

## Logistic Regression

In [ ]:
Precisions = []
Recalls = []
F1s = []
G_means = []
accuracy = []
AUC = []
Specificities = []

for i in range(500):
  cv = StratifiedKFold(n_splits=5, shuffle=True)
  for train_fold_index, val_fold_index in cv.split(X,Y):
    X_train_fold,y_train_fold = X[train_fold_index], Y[train_fold_index]
    X_val_fold, y_val_fold = X[val_fold_index], Y[val_fold_index]
    model = LogisticRegression(class_weight = "balanced")
    model.fit(X_train_fold,y_train_fold)
    pt = model.predict(X_val_fold)
    tn, fp, fn, tp = confusion_matrix(y_val_fold, pt).ravel()
    specificity = tn / (tn+fp)
    Specificities.append(specificity)
    Precisions.append(precision_score(y_val_fold,pt))
    Recalls.append(recall_score(y_val_fold,pt))
    F1s.append(f1_score(y_val_fold,pt))
    G_means.append(geometric_mean_score(y_val_fold,pt))
    accuracy.append(accuracy_score(y_val_fold,pt))
    AUC.append(roc_auc_score(y_val_fold,pt))

print('Precision- Mean:  %.3f Standard Deviation: %.3f' % (mean(Precisions), statistics.pstdev(Precisions)))
print('Sensitivity/Recall- Mean:  %.3f Standard Deviation: %.3f' % (mean(Recalls), statistics.pstdev(Recalls)))
print('Specificity - Mean:  %.3f Standard Deviation: %.3f' % (mean(Specificities), statistics.pstdev(Specificities)))
print('F1- Mean:  %.3f Standard Deviation: %.3f' % (mean(F1s), statistics.pstdev(F1s)))
print('G_mean- Mean:  %.3f Standard Deviation: %.3f' % (mean(G_means), statistics.pstdev(G_means)))
print('Accuracy- Mean:  %.3f Standard Deviation: %.3f' % (mean(accuracy), statistics.pstdev(accuracy)))
print('AUC Score- Mean:  %.3f Standard Deviation: %.3f' % (mean(AUC), statistics.pstdev(AUC)))

## MLP

### "Wide"

In [ ]:
Precisions = []
Recalls = []
F1s = []
G_means = []
accuracy = []
AUC = []
Specificities = []

for i in range(500):
  cv = StratifiedKFold(n_splits=5, shuffle=True)
  for train_fold_index, val_fold_index in cv.split(X,Y):
    X_train_fold_resample,y_train_fold_resample = X[train_fold_index], Y[train_fold_index]
    X_val_fold, y_val_fold = X[val_fold_index], Y[val_fold_index]
    model = MLPClassifier(hidden_layer_sizes=(512, 512, 512), random_state=1)
    model.fit(X_train_fold_resample,y_train_fold_resample)
    pt = model.predict(X_val_fold)
    tn, fp, fn, tp = confusion_matrix(y_val_fold, pt).ravel()
    specificity = tn / (tn+fp)
    Specificities.append(specificity)
    Precisions.append(precision_score(y_val_fold,pt))
    Recalls.append(recall_score(y_val_fold,pt))
    F1s.append(f1_score(y_val_fold,pt))
    G_means.append(geometric_mean_score(y_val_fold,pt))
    accuracy.append(accuracy_score(y_val_fold,pt))
    AUC.append(roc_auc_score(y_val_fold,pt))

print('Precision- Mean:  %.3f Standard Deviation: %.3f' % (mean(Precisions), statistics.pstdev(Precisions)))
print('Sensitivity/Recall- Mean:  %.3f Standard Deviation: %.3f' % (mean(Recalls), statistics.pstdev(Recalls)))
print('Specificity - Mean:  %.3f Standard Deviation: %.3f' % (mean(Specificities), statistics.pstdev(Specificities)))
print('F1- Mean:  %.3f Standard Deviation: %.3f' % (mean(F1s), statistics.pstdev(F1s)))
print('G_mean- Mean:  %.3f Standard Deviation: %.3f' % (mean(G_means), statistics.pstdev(G_means)))
print('Accuracy- Mean:  %.3f Standard Deviation: %.3f' % (mean(accuracy), statistics.pstdev(accuracy)))
print('AUC Score- Mean:  %.3f Standard Deviation: %.3f' % (mean(AUC), statistics.pstdev(AUC)))

In [ ]:
# p value 

cv = StratifiedKFold(n_splits=5, random_state=12, shuffle=True)
AUC_metric = make_scorer(roc_auc_score)
g_mean_metric = make_scorer(geometric_mean_score)
p_values_AUC = {}
p_values_g_mean = {}
titles = ["AUC p-value", "G-Mean p-value"]
model = MLPClassifier(hidden_layer_sizes=(512, 512, 512), random_state=1, tol = 0.005)

_, _, pvalue = permutation_test_score(model, X, Y, scoring=AUC_metric, cv=cv, n_permutations=1000)
p_values_AUC["auc"] = pvalue
print(p_values_AUC)
_, _, pvalue = permutation_test_score(model, X, Y, scoring=g_mean_metric, cv=cv, n_permutations=1000)
p_values_g_mean["gmean"] = pvalue

print(p_values_g_mean)

### "Deep"

In [ ]:
Precisions = []
Recalls = []
F1s = []
G_means = []
accuracy = []
AUC = []
Specificities = []

for i in range(500):
  cv = StratifiedKFold(n_splits=5, shuffle=True)
  for train_fold_index, val_fold_index in cv.split(X,Y):
    X_train_fold_resample,y_train_fold_resample = X[train_fold_index], Y[train_fold_index]
    X_val_fold, y_val_fold = X[val_fold_index], Y[val_fold_index]
    model = MLPClassifier(hidden_layer_sizes=(100,100,100,100,100,100,100,100,100,100), random_state=1)
    model.fit(X_train_fold_resample,y_train_fold_resample)
    pt = model.predict(X_val_fold)
    tn, fp, fn, tp = confusion_matrix(y_val_fold, pt).ravel()
    specificity = tn / (tn+fp)
    Specificities.append(specificity)
    Precisions.append(precision_score(y_val_fold,pt))
    Recalls.append(recall_score(y_val_fold,pt))
    F1s.append(f1_score(y_val_fold,pt))
    G_means.append(geometric_mean_score(y_val_fold,pt))
    accuracy.append(accuracy_score(y_val_fold,pt))
    AUC.append(roc_auc_score(y_val_fold,pt))

print('Precision- Mean:  %.3f Standard Deviation: %.3f' % (mean(Precisions), statistics.pstdev(Precisions)))
print('Sensitivity/Recall- Mean:  %.3f Standard Deviation: %.3f' % (mean(Recalls), statistics.pstdev(Recalls)))
print('Specificity - Mean:  %.3f Standard Deviation: %.3f' % (mean(Specificities), statistics.pstdev(Specificities)))
print('F1- Mean:  %.3f Standard Deviation: %.3f' % (mean(F1s), statistics.pstdev(F1s)))
print('G_mean- Mean:  %.3f Standard Deviation: %.3f' % (mean(G_means), statistics.pstdev(G_means)))
print('Accuracy- Mean:  %.3f Standard Deviation: %.3f' % (mean(accuracy), statistics.pstdev(accuracy)))
print('AUC Score- Mean:  %.3f Standard Deviation: %.3f' % (mean(AUC), statistics.pstdev(AUC)))

In [ ]:
# p value 

cv = StratifiedKFold(n_splits=5, random_state=12, shuffle=True)
AUC_metric = make_scorer(roc_auc_score)
g_mean_metric = make_scorer(geometric_mean_score)
p_values_AUC = {}
p_values_g_mean = {}
titles = ["AUC p-value", "G-Mean p-value"]
model = MLPClassifier(hidden_layer_sizes=(100,100,100,100,100,100,100,100,100,100), random_state=1)

_, _, pvalue = permutation_test_score(model, X, Y, scoring=AUC_metric, cv=cv, n_permutations=1000)
p_values_AUC["auc"] = pvalue
_, _, pvalue = permutation_test_score(model, X, Y, scoring=g_mean_metric, cv=cv, n_permutations=1000)
p_values_g_mean["gmean"] = pvalue

print(p_values_AUC)
print(p_values_g_mean)

### "Pyramid"

In [ ]:

Precisions = []
Recalls = []
F1s = []
G_means = []
accuracy = []
AUC = []
Specificities = []

for i in range(500):
  cv = StratifiedKFold(n_splits=5, shuffle=True)
  for train_fold_index, val_fold_index in cv.split(X,Y):
    X_train_fold_resample,y_train_fold_resample = X[train_fold_index], Y[train_fold_index]
    X_val_fold, y_val_fold = X[val_fold_index], Y[val_fold_index]
    model = MLPClassifier(hidden_layer_sizes=(512, 256, 128, 64, 64), random_state=1, max_iter = 400)
    model.fit(X_train_fold_resample,y_train_fold_resample)
    pt = model.predict(X_val_fold)

    tn, fp, fn, tp = confusion_matrix(y_val_fold, pt).ravel()
    specificity = tn / (tn+fp)
    Specificities.append(specificity)
    Precisions.append(precision_score(y_val_fold,pt))
    Recalls.append(recall_score(y_val_fold,pt))
    F1s.append(f1_score(y_val_fold,pt))
    G_means.append(geometric_mean_score(y_val_fold,pt))
    accuracy.append(accuracy_score(y_val_fold,pt))
    AUC.append(roc_auc_score(y_val_fold,pt))

print('Precision- Mean:  %.3f Standard Deviation: %.3f' % (mean(Precisions), statistics.pstdev(Precisions)))
print('Sensitivity/Recall- Mean:  %.3f Standard Deviation: %.3f' % (mean(Recalls), statistics.pstdev(Recalls)))
print('Specificity - Mean:  %.3f Standard Deviation: %.3f' % (mean(Specificities), statistics.pstdev(Specificities)))
print('F1- Mean:  %.3f Standard Deviation: %.3f' % (mean(F1s), statistics.pstdev(F1s)))
print('G_mean- Mean:  %.3f Standard Deviation: %.3f' % (mean(G_means), statistics.pstdev(G_means)))
print('Accuracy- Mean:  %.3f Standard Deviation: %.3f' % (mean(accuracy), statistics.pstdev(accuracy)))
print('AUC Score- Mean:  %.3f Standard Deviation: %.3f' % (mean(AUC), statistics.pstdev(AUC)))

In [ ]:
# p value 

from sklearn.neural_network import MLPClassifier
cv = StratifiedKFold(n_splits=5, random_state=12, shuffle=True)
AUC_metric = make_scorer(roc_auc_score)
g_mean_metric = make_scorer(geometric_mean_score)
p_values_AUC = {}
p_values_g_mean = {}
titles = ["AUC p-value", "G-Mean p-value"]
model = MLPClassifier(hidden_layer_sizes=(512, 256, 128, 64, 64), random_state=1, max_iter = 400)
_, _, pvalue = permutation_test_score(model, X, Y, scoring=AUC_metric, cv=cv, n_permutations=1000)
p_values_AUC["auc"] = pvalue

print(p_values_AUC)
_, _, pvalue = permutation_test_score(model, X, Y, scoring=g_mean_metric, cv=cv, n_permutations=1000)
p_values_g_mean["gmean"] = pvalue

print(p_values_g_mean)

## KNN

In [ ]:
for j in [3,5,7,9,11]:
  Precisions = []
  Recalls = []
  F1s = []
  G_means = []
  accuracy = []
  AUC = []
  Specificities = []

  for i in range(500):
    cv = StratifiedKFold(n_splits=5, shuffle=True)
    for train_fold_index, val_fold_index in cv.split(X,Y):
      X_train_fold_resample,y_train_fold_resample = X[train_fold_index], Y[train_fold_index]
      X_val_fold, y_val_fold = X[val_fold_index], Y[val_fold_index]
      model = KNeighborsClassifier(n_neighbors=j)

      model.fit(X_train_fold_resample,y_train_fold_resample)
      pt = model.predict(X_val_fold)
      tn, fp, fn, tp = confusion_matrix(y_val_fold, pt).ravel()
      specificity = tn / (tn+fp)
      Specificities.append(specificity)
      Precisions.append(precision_score(y_val_fold,pt))
      Recalls.append(recall_score(y_val_fold,pt))
      F1s.append(f1_score(y_val_fold,pt))
      G_means.append(geometric_mean_score(y_val_fold,pt))
      accuracy.append(accuracy_score(y_val_fold,pt))
      AUC.append(roc_auc_score(y_val_fold,pt))
  print("k = "+ str(j))
  print('Precision- Mean:  %.3f Standard Deviation: %.3f' % (mean(Precisions), statistics.pstdev(Precisions)))
  print('Sensitivity/Recall- Mean:  %.3f Standard Deviation: %.3f' % (mean(Recalls), statistics.pstdev(Recalls)))
  print('Specificity - Mean:  %.3f Standard Deviation: %.3f' % (mean(Specificities), statistics.pstdev(Specificities)))
  print('F1- Mean:  %.3f Standard Deviation: %.3f' % (mean(F1s), statistics.pstdev(F1s)))
  print('G_mean- Mean:  %.3f Standard Deviation: %.3f' % (mean(G_means), statistics.pstdev(G_means)))
  print('Accuracy- Mean:  %.3f Standard Deviation: %.3f' % (mean(accuracy), statistics.pstdev(accuracy)))
  print('AUC Score- Mean:  %.3f Standard Deviation: %.3f' % (mean(AUC), statistics.pstdev(AUC)))

## XGBoost

In [ ]:
Precisions = []
Recalls = []
Specificities = []
F1s = []
G_means = []
accuracy = []
AUC = []

for i in range(500):
  cv = StratifiedKFold(n_splits=5, shuffle=True)
  for train_fold_index, val_fold_index in cv.split(X,Y):
    X_train_fold,y_train_fold = X[train_fold_index], Y[train_fold_index]
    X_val_fold, y_val_fold = X[val_fold_index], Y[val_fold_index]
    model = XGBClassifier(max_depth=4, n_estimators=5, scale_pos_weight=1.44)
    model.fit(X_train_fold,y_train_fold)
    pt = model.predict(X_val_fold)
    tn, fp, fn, tp = confusion_matrix(y_val_fold, pt).ravel()
    Specificities.append(tn / (tn+fp))
    Precisions.append(precision_score(y_val_fold,pt))
    Recalls.append(recall_score(y_val_fold,pt))
    F1s.append(f1_score(y_val_fold,pt))
    G_means.append(geometric_mean_score(y_val_fold,pt))
    accuracy.append(accuracy_score(y_val_fold,pt))
    AUC.append(roc_auc_score(y_val_fold,pt))

print('Precision - Mean:  %.3f Standard Deviation: %.3f' % (mean(Precisions), statistics.pstdev(Precisions)))
print('Sensitivity/Recall - Mean:  %.3f Standard Deviation: %.3f' % (mean(Recalls), statistics.pstdev(Recalls)))
print('Specificity - Mean:  %.3f Standard Deviation: %.3f' % (mean(Specificities), statistics.pstdev(Specificities)))
print('F1- Mean:  %.3f Standard Deviation: %.3f' % (mean(F1s), statistics.pstdev(F1s)))
print('G_mean- Mean:  %.3f Standard Deviation: %.3f' % (mean(G_means), statistics.pstdev(G_means)))
print('Accuracy- Mean:  %.3f Standard Deviation: %.3f' % (mean(accuracy), statistics.pstdev(accuracy)))
print('AUC Score- Mean:  %.3f Standard Deviation: %.3f' % (mean(AUC), statistics.pstdev(AUC)))

## XGBoost with Undersampling

In [ ]:
Precisions = []
Recalls = []
F1s = []
G_means = []
accuracy = []
AUC = []

for i in range(500):
  cv = StratifiedKFold(n_splits=5, shuffle=True)
  for train_fold_index, val_fold_index in cv.split(X,Y):
    X_train_fold,y_train_fold = X[train_fold_index], Y[train_fold_index]
    X_val_fold, y_val_fold = X[val_fold_index], Y[val_fold_index]
    model = XGBClassifier(n_estimators=180, max_depth=4, scale_pos_weight=1.5)
    model.fit(X_train_fold,y_train_fold)
    pt = model.predict(X_val_fold)
    Precisions.append(precision_score(y_val_fold,pt))
    Recalls.append(recall_score(y_val_fold,pt))
    F1s.append(f1_score(y_val_fold,pt))
    G_means.append(geometric_mean_score(y_val_fold,pt))
    accuracy.append(accuracy_score(y_val_fold,pt))
    AUC.append(roc_auc_score(y_val_fold,pt))

print('Precision- Mean:  %f Standard Deviation: %f' % (mean(Precisions), statistics.pstdev(Precisions)))
print('Recall- Mean:  %f Standard Deviation: %f' % (mean(Recalls), statistics.pstdev(Recalls)))
print('F1- Mean:  %f Standard Deviation: %f' % (mean(F1s), statistics.pstdev(F1s)))
print('G_mean- Mean:  %f Standard Deviation: %f' % (mean(G_means), statistics.pstdev(G_means)))
print('Accuracy- Mean:  %f Standard Deviation: %f' % (mean(accuracy), statistics.pstdev(accuracy)))
print('AUC Score- Mean:  %f Standard Deviation: %f' % (mean(AUC), statistics.pstdev(AUC)))

## XGBoost with Oversampling

### SMOTE


In [ ]:
Precisions = []
Recalls = []
F1s = []
G_means = []
accuracy = []
AUC = []
Specificities = []

for i in range(500):
  cv = StratifiedKFold(n_splits=5, shuffle=True)
  for train_fold_index, val_fold_index in cv.split(X,Y):
    X_train_fold,y_train_fold = X[train_fold_index], Y[train_fold_index]
    X_val_fold, y_val_fold = X[val_fold_index], Y[val_fold_index]
    smoter = SMOTE()
    X_train_fold_resample, y_train_fold_resample = smoter.fit_resample(X_train_fold,y_train_fold)
    model = XGBClassifier(max_depth=5, n_estimators=9, scale_pos_weight=2.47)
    model.fit(X_train_fold_resample,y_train_fold_resample)
    pt = model.predict(X_val_fold)

    tn, fp, fn, tp = confusion_matrix(y_val_fold, pt).ravel()
    specificity = tn / (tn+fp)
    Specificities.append(specificity)
    Precisions.append(precision_score(y_val_fold,pt))
    Recalls.append(recall_score(y_val_fold,pt))
    F1s.append(f1_score(y_val_fold,pt))
    G_means.append(geometric_mean_score(y_val_fold,pt))
    accuracy.append(accuracy_score(y_val_fold,pt))
    AUC.append(roc_auc_score(y_val_fold,pt))

print('Precision- Mean:  %.3f Standard Deviation: %.3f' % (mean(Precisions), statistics.pstdev(Precisions)))
print('Sensitivity/Recall- Mean:  %.3f Standard Deviation: %.3f' % (mean(Recalls), statistics.pstdev(Recalls)))
print('Specificity - Mean:  %.3f Standard Deviation: %.3f' % (mean(Specificities), statistics.pstdev(Specificities)))
print('F1- Mean:  %.3f Standard Deviation: %.3f' % (mean(F1s), statistics.pstdev(F1s)))
print('G_mean- Mean:  %.3f Standard Deviation: %.3f' % (mean(G_means), statistics.pstdev(G_means)))
print('Accuracy- Mean:  %.3f Standard Deviation: %.3f' % (mean(accuracy), statistics.pstdev(accuracy)))
print('AUC Score- Mean:  %.3f Standard Deviation: %.3f' % (mean(AUC), statistics.pstdev(AUC)))

### Random Oversampling

In [ ]:
Precisons = []
Recalls = []
F1s = []
G_means = []
accuracy = []
AUC = []
for i in range(500):
  cv = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
  for train_fold_index, val_fold_index in cv.split(X,Y):
    X_train_fold,y_train_fold = X[train_fold_index], Y[train_fold_index]
    X_val_fold, y_val_fold = X[val_fold_index], Y[val_fold_index]
    ros3 = RandomOverSampler(random_state=0)
    X_train_fold_resample, y_train_fold_resample = ros3.fit_resample(X_train_fold,y_train_fold)
    model = XGBClassifier(n_estimators=9, max_depth=5, scale_pos_weight= 2.47)
    model.fit(X_train_fold_resample,y_train_fold_resample)
    pt = model.predict(X_val_fold)

    Precisons.append(precision_score(y_val_fold,pt))
    Recalls.append(recall_score(y_val_fold,pt))
    F1s.append(f1_score(y_val_fold,pt))
    G_means.append(geometric_mean_score(y_val_fold,pt))
    accuracy.append(accuracy_score(y_val_fold,pt))
    AUC.append(roc_auc_score(y_val_fold,pt))

print('Precision- Mean:  %.3f Standard Deviation: %.3f' % (mean(Precisions), statistics.pstdev(Precisions)))
print('Recall- Mean:  %.3f Standard Deviation: %.3f' % (mean(Recalls), statistics.pstdev(Recalls)))
print('F1- Mean:  %.3f Standard Deviation: %.3f' % (mean(F1s), statistics.pstdev(F1s)))
print('G_mean- Mean:  %.3f Standard Deviation: %.3f' % (mean(G_means), statistics.pstdev(G_means)))
print('Accuracy- Mean:  %.3f Standard Deviation: %.3f' % (mean(accuracy), statistics.pstdev(accuracy)))
print('AUC Score- Mean:  %.3f Standard Deviation: %.3f' % (mean(AUC), statistics.pstdev(AUC)))

# Naive Classifiers

## Majority Classifier Metrics


In [ ]:
# Naive Classifier 
## Predicts the Majority (Mucinous) Class
## Source: https://machinelearningmastery.com/how-to-develop-and-evaluate-naive-classifier-strategies-using-probability/
##         https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics 

from scipy.stats import mode
from sklearn.metrics import recall_score, precision_score, f1_score,accuracy_score
from imblearn.metrics import geometric_mean_score

# predict the majority class
def majority_class(y):
	return mode(Y)[0]

# make predictions
yhat = [0 for _ in range(len(Y))]
print(yhat)
tn, fp, fn, tp = confusion_matrix(Y, yhat).ravel()

# calculate Metrics
print('F1 : %.3f' % f1_score(Y, yhat))
print('Recall : %.3f' % recall_score(Y,yhat))
print('Specificity :  %.3f' % (tn/(tn+fp)))
print('Precision : %.3f' % precision_score(Y,yhat))
print('G-Mean : %.3f' % geometric_mean_score(Y,yhat))
print('accuracy : %.3f' % accuracy_score(Y,yhat))
print('AUC:  %.3f' % roc_auc_score(Y,yhat))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
F1 : 0.000
Recall : 0.000
Specificity :  1.000
Precision : 0.000
G-Mean : 0.000
accuracy : 0.675
AUC:  0.500


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Minority Classifier

In [ ]:
# Naive Classifier 
## Predicts the Majority (Mucinous) Class
## Source: https://machinelearningmastery.com/how-to-develop-and-evaluate-naive-classifier-strategies-using-probability/
##         https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics 


# predict the majority class
def majority_class(Y):
	return mode(y)[0]

# make predictions
yhat = [1 for _ in range(len(Y))] #Hardcoded for our model's distribution
print(yhat)
tn, fp, fn, tp = confusion_matrix(Y, yhat).ravel()

# calculate Metrics
print('F1 : %.3f' % f1_score(Y, yhat))
print('Recall/sensitivity : %.3f' % recall_score(Y,yhat))
print('Specificity :  %.3f' % (tn/(tn+fp)))
print('Precision : %.3f' % precision_score(Y,yhat))
print('G-Mean : %.3f' % geometric_mean_score(Y,yhat))
print('accuracy : %.3f' % accuracy_score(Y,yhat))
print('AUC:  %.3f' % roc_auc_score(Y,yhat))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
F1 : 0.491
Recall/sensitivity : 1.000
Specificity :  0.000
Precision : 0.325
G-Mean : 0.000
accuracy : 0.325
AUC:  0.500


## Random Guesser

In [ ]:
dummy_clf = DummyClassifier(strategy="uniform")
dummy_clf.fit(X, Y)
y_predicted = dummy_clf.predict(X)

f1= []
rcll = []
prc = []
gmean = []
acc = []
spec = []
roc = []

for i in range(10000):
  y_predicted = dummy_clf.predict(X)
  f1.append(f1_score(Y, y_predicted))
  rcll.append(recall_score(Y,y_predicted))
  prc.append(precision_score(Y,y_predicted))
  gmean.append(geometric_mean_score(Y,y_predicted))
  acc.append(accuracy_score(Y,y_predicted))
  tn, fp, fn, tp = confusion_matrix(Y, y_predicted).ravel()
  spec.append(tn/(tn+fp))
  roc.append(roc_auc_score(Y,y_predicted))

print('Precision - Mean:  %.3f Standard Deviation: %.3f' % (mean(prc), statistics.pstdev(prc)))
print('Sensitivity/Recall - Mean:  %.3f Standard Deviation: %.3f' % (mean(rcll), statistics.pstdev(rcll)))
print('Specificity - Mean:  %.3f Standard Deviation: %.3f' % (mean(spec), statistics.pstdev(spec)))
print('F1- Mean:  %.3f Standard Deviation: %.3f' % (mean(f1), statistics.pstdev(f1)))
print('G_mean- Mean:  %.3f Standard Deviation: %.3f' % (mean(gmean), statistics.pstdev(gmean)))
print('Accuracy- Mean:  %.3f Standard Deviation: %.3f' % (mean(acc), statistics.pstdev(acc)))
print('AUC Score- Mean:  %.3f Standard Deviation: %.3f' % (mean(roc), statistics.pstdev(roc)))

Precision - Mean:  0.325 Standard Deviation: 0.053
Sensitivity/Recall - Mean:  0.500 Standard Deviation: 0.098
Specificity - Mean:  0.501 Standard Deviation: 0.068
F1- Mean:  0.393 Standard Deviation: 0.066
G_mean- Mean:  0.497 Standard Deviation: 0.060
Accuracy- Mean:  0.501 Standard Deviation: 0.056
AUC Score- Mean:  0.500 Standard Deviation: 0.060


#ALL model Pvalues


## P values for various models


In [ ]:
## Full Feature Set
df = pd.read_csv(DATASET_LOCATION)
x_texture = df.drop("hgd_malignancy", axis=1);
Y_texture = df["hgd_malignancy"].copy();
feature_cols = x_texture.columns
scaler = MinMaxScaler(feature_range=(0, 1))
X_texture = scaler.fit_transform(x_texture)

## Non-texture only Feature Set
df = pd.read_csv(DATASET_LOCATION)
x_combined = df.drop("hgd_malignancy", axis=1);
Y_combined  = df["hgd_malignancy"].copy();
feature_cols = x_combined.columns
scaler = MinMaxScaler(feature_range=(0, 1))
X_combined = scaler.fit_transform(x_combined)

# Models
linear = LogisticRegression(class_weight = "balanced")
svc = svm.SVC()
rf = RandomForestClassifier(n_estimators=25,max_depth=20, class_weight='balanced')
knn3 = KNeighborsClassifier(n_neighbors=3)
knn5 = KNeighborsClassifier(n_neighbors=5)
knn7 = KNeighborsClassifier(n_neighbors=7)
knn9 = KNeighborsClassifier(n_neighbors=9)
knn11 = KNeighborsClassifier(n_neighbors=11)

## Setup
cv = StratifiedKFold(n_splits=5, shuffle=True)
AUC_metric = make_scorer(roc_auc_score)
g_mean_metric = make_scorer(geometric_mean_score)
p_values_AUC = {}
p_values_g_mean = {}
titles = ["AUC p-value", "G-Mean p-value"]

## AUC
_, _, pvalue = permutation_test_score(linear, X_combined, Y_combined, scoring=AUC_metric, cv=cv, n_permutations=1000)
p_values_AUC["linear"] = pvalue
_, _, pvalue = permutation_test_score(svc, X_combined, Y_combined, scoring=AUC_metric, cv=cv, n_permutations=1000)
p_values_AUC["svc"] = pvalue
_, _, pvalue = permutation_test_score(knn3, X_combined, Y_combined, scoring=AUC_metric, cv=cv, n_permutations=1000)
p_values_AUC["knn3"] = pvalue
_, _, pvalue = permutation_test_score(knn5, X_combined, Y_combined, scoring=AUC_metric, cv=cv, n_permutations=1000)
p_values_AUC["knn5"] = pvalue
_, _, pvalue = permutation_test_score(knn7, X_combined, Y_combined, scoring=AUC_metric, cv=cv, n_permutations=1000)
p_values_AUC["knn7"] = pvalue
_, _, pvalue = permutation_test_score(knn9, X_combined, Y_combined, scoring=AUC_metric, cv=cv, n_permutations=1000)
p_values_AUC["knn9"] = pvalue
_, _, pvalue = permutation_test_score(knn11, X_combined, Y_combined, scoring=AUC_metric, cv=cv, n_permutations=1000)
p_values_AUC["knn11"] = pvalue
_, _, pvalue = permutation_test_score(rf, X_combined, Y_combined, scoring=AUC_metric, cv=cv, n_permutations=1000)
p_values_AUC["rf"] = pvalue

# G - Mean
_, _, pvalue = permutation_test_score(linear, X_combined, Y_combined, scoring=g_mean_metric, cv=cv, n_permutations=1000)
p_values_g_mean["linear"] = pvalue
_, _, pvalue = permutation_test_score(svc, X_combined, Y_combined, scoring=g_mean_metric, cv=cv, n_permutations=1000)
p_values_g_mean["svc"] = pvalue
_, _, pvalue = permutation_test_score(knn3, X_combined, Y_combined, scoring=g_mean_metric, cv=cv, n_permutations=1000)
p_values_g_mean["knn3"] = pvalue
_, _, pvalue = permutation_test_score(knn5, X_combined, Y_combined, scoring=g_mean_metric, cv=cv, n_permutations=1000)
p_values_g_mean["knn5"] = pvalue
_, _, pvalue = permutation_test_score(knn7, X_combined, Y_combined, scoring=g_mean_metric, cv=cv, n_permutations=1000)
p_values_g_mean["knn7"] = pvalue
_, _, pvalue = permutation_test_score(knn9, X_combined, Y_combined, scoring=g_mean_metric, cv=cv, n_permutations=1000)
p_values_g_mean["knn9"] = pvalue
_, _, pvalue = permutation_test_score(knn11, X_combined, Y_combined, scoring=g_mean_metric, cv=cv, n_permutations=1000)
p_values_g_mean["knn11"] = pvalue
_, _, pvalue = permutation_test_score(rf, X_combined, Y_combined, scoring=g_mean_metric, cv=cv, n_permutations=1000)
p_values_g_mean["rf"] = pvalue

# Output Table
print("AUC")
print(p_values_AUC)
print("G-Mean")
print(p_values_g_mean)


## P values for XGBoost and Naive classifiers


In [ ]:
# Datasets
## Full Feature Set
df = pd.read_csv(DATASET_LOCATION)
x_full = df.drop("hgd_malignancy", axis=1); #Entire dataset
Y_full = df["hgd_malignancy"].copy()
scaler = MinMaxScaler(feature_range=(0, 1))
X_full = scaler.fit_transform(x_full)

#Import Texture-Only Feature Set
df = pd.read_csv(DATASET_LOCATION)
x_texture = df.drop("hgd_malignancy", axis=1); #Entire dataset
Y_texture = df["hgd_malignancy"].copy();
scaler = MinMaxScaler(feature_range=(0, 1))
X_texture = scaler.fit_transform(x_texture)

# Models
## Naive
## Majority, Minority, random, stratified
majority = DummyClassifier(strategy='constant', constant=1) #strategy='most_frequent'
minority = DummyClassifier(strategy='constant', constant=0)
random = DummyClassifier(strategy='uniform', constant=1)
stratified = DummyClassifier(strategy='stratified', constant=1)
random.fit(X_full, Y_full)
stratified.fit(X_full, Y_full)

## ML
## SMOTE Full Feature, SMOTE Texture-Only, XGBoost Full, XGBoost Texture-only
XGBoost = XGBClassifier(n_estimators=11, max_depth=3, scale_pos_weight=.25)
SMOTE_XGBoost = Pipeline([
        ('sampling', SMOTE()),
        ('classification', XGBoost)
    ])

# Scoring
## Setup
cv = StratifiedKFold(n_splits=5, random_state=12, shuffle=True)
AUC_metric = make_scorer(roc_auc_score)
g_mean_metric = make_scorer(geometric_mean_score)
p_values_AUC = {}
p_values_g_mean = {}
titles = ["AUC p-value", "G-Mean p-value"]

## AUC
_, _, pvalue = permutation_test_score(majority, X_full, Y_full, scoring=AUC_metric, cv=cv, n_permutations=1000)
p_values_AUC["majority"] = pvalue
_, _, pvalue = permutation_test_score(minority, X_full, Y_full, scoring=AUC_metric, cv=cv, n_permutations=1000)
p_values_AUC["minority"] = pvalue
_, _, pvalue = permutation_test_score(random, X_full, Y_full, scoring=AUC_metric, cv=cv, n_permutations=1000)
p_values_AUC["random"] = pvalue
_, _, pvalue = permutation_test_score(stratified, X_full, Y_full, scoring=AUC_metric, cv=cv, n_permutations=1000)
p_values_AUC["stratified"] = pvalue
_, _, pvalue = permutation_test_score(XGBoost, X_full, Y_full, scoring=AUC_metric, cv=cv, n_permutations=1000)
p_values_AUC["XGBoost_Full"] = pvalue
_, _, pvalue = permutation_test_score(SMOTE_XGBoost, X_full, Y_full, scoring=AUC_metric, cv=cv, n_permutations=1000)
p_values_AUC["SMOTE_Full"] = pvalue
_, _, pvalue = permutation_test_score(XGBoost, X_texture, Y_texture, scoring=AUC_metric, cv=cv, n_permutations=1000)
p_values_AUC["XGBoost_Texture"] = pvalue
_, _, pvalue = permutation_test_score(SMOTE_XGBoost, X_texture, Y_texture, scoring=AUC_metric, cv=cv, n_permutations=1000)
p_values_AUC["SMOTE_Texture"] = pvalue

## G - Mean
_, _, pvalue = permutation_test_score(majority, X_full, Y_full, scoring=g_mean_metric, cv=cv, n_permutations=1000)
p_values_g_mean["majority"] = pvalue
_, _, pvalue = permutation_test_score(minority, X_full, Y_full, scoring=g_mean_metric, cv=cv, n_permutations=1000)
p_values_g_mean["minority"] = pvalue
_, _, pvalue = permutation_test_score(random, X_full, Y_full, scoring=g_mean_metric, cv=cv, n_permutations=1000)
p_values_g_mean["random"] = pvalue
_, _, pvalue = permutation_test_score(stratified, X_full, Y_full, scoring=g_mean_metric, cv=cv, n_permutations=1000)
p_values_g_mean["stratified"] = pvalue
score, _, pvalue = permutation_test_score(XGBoost, X_full, Y_full, scoring=g_mean_metric, cv=cv, n_permutations=1000)
p_values_g_mean["XGBoost_Full"] = pvalue
_, _, pvalue = permutation_test_score(SMOTE_XGBoost, X_full, Y_full, scoring=g_mean_metric, cv=cv, n_permutations=1000)
p_values_g_mean["SMOTE_Full"] = pvalue
_, _, pvalue = permutation_test_score(XGBoost, X_texture, Y_texture, scoring=g_mean_metric, cv=cv, n_permutations=1000)
p_values_g_mean["XGBoost_Texture"] = pvalue
_, _, pvalue = permutation_test_score(SMOTE_XGBoost, X_texture, Y_texture, scoring=g_mean_metric, cv=cv, n_permutations=1000)
p_values_g_mean["SMOTE_Texture"] = pvalue

# Output Table
print("AUC")
print(p_values_AUC)
print("G-Mean")
print(p_values_g_mean)

# Curves

## PR Curves

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning) 

import matplotlib.pyplot as plt
import numpy
from sklearn.datasets import make_blobs
from sklearn.metrics import precision_recall_curve, auc
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from numpy import interp
from xgboost import XGBClassifier

FOLDS = 10

f, axes = plt.subplots(figsize=(10,10))
k_fold = StratifiedKFold(n_splits=FOLDS, random_state=12, shuffle=True)
results = pd.DataFrame(columns=['training_score', 'test_score'])


y_realtot = []
y_probatot = []

precision_arraytot = []
threshold_arraytot=[]
recall_arraytot = np.linspace(0, 1, 100)

for j in range(10):
  y_real = []
  y_proba = []

  precision_array = []
  threshold_array=[]
  recall_array = np.linspace(0, 1, 100)
  for i, (train_index, test_index) in enumerate(k_fold.split(X,Y)):
    predictor = XGBClassifier(n_estimators=45, max_depth=4, scale_pos_weight=55)
      
    X_train_fold,y_train_fold = X[train_index], Y[train_index]
    X_val_fold, y_val_fold = X[test_index], Y[test_index]
    smoter = SMOTE(random_state=12)
    X_train_fold_resample, y_train_fold_resample = smoter.fit_resample(X_train_fold,y_train_fold)
      
    predictor.fit(X_train_fold_resample, y_train_fold_resample)
    pred_proba = predictor.predict_proba(X_val_fold)
    precision_fold, recall_fold, thresh = precision_recall_curve(y_val_fold, pred_proba[:,1])
    precision_fold, recall_fold, thresh = precision_fold[::-1], recall_fold[::-1], thresh[::-1]  # reverse order of results
    thresh = np.insert(thresh, 0, 1.0)
    precision_array = interp(recall_array, recall_fold, precision_fold)
    threshold_array = interp(recall_array, recall_fold, thresh)
    pr_auc = auc(recall_array, precision_array)

    lab_fold = 'Fold %d AUC=%.4f' % (i+1, pr_auc)
    y_real.append(y_val_fold)
    y_proba.append(pred_proba[:,1])

  y_real = numpy.concatenate(y_real)
  y_proba = numpy.concatenate(y_proba)
  precision, recall, _ = precision_recall_curve(y_real, y_proba)
  lab_foldtot = 'PR %d AUC=%.4f' % (j+1, pr_auc)
  plt.plot(recall, precision, marker='.' ,alpha=0.3, label=lab_foldtot)
  y_realtot.append(y_real)
  y_probatot.append(y_proba)
  precision_arraytot = interp(recall_array, recall, precision)
  threshold_arraytot = interp(recall_array, recall, precision)

#finsih 10 iterations.
y_realtot = numpy.concatenate(y_realtot)
y_probatot= numpy.concatenate(y_probatot)
precision, recall, _ = precision_recall_curve(y_realtot, y_probatot)
lab = 'Overall AUC=%.4f' % (auc(recall, precision))

plt.plot(recall, precision, marker='.', lw=2,color='red', label=lab)
plt.legend(loc='lower left', fontsize=18)

lab = 'Overall AUC=%.4f' % (auc(recall, precision))
mean_precision = np.mean(precision)
mean_recall = np.mean(recall)
std_precision = np.std(precision)
print ("mean of precision: " )
print (mean_precision )

print ("Std Dev of precision: ")
print ( std_precision )

axes.set_title('10 Indenpendent PR Curves of Random Forest Over 10 Folds Cross Validation', fontsize=18)

plt.fill_between(recall, precision + std_precision, precision - std_precision, alpha=0.3, linewidth=0, color='grey')
plt.xlabel("Recall", fontsize=18)
plt.ylabel("Precision", fontsize=18)
plt.ylim((0,1))
plt.xlim((0,1))
plt.show()

f.savefig('result.png')
print (precision)
print (recall)
print (_)

## ROC

In [ ]:
 ## ROC Curve for 10-Fold Cross Validation with SMOTE oversampling 
 # Source: https://ogrisel.github.io/scikit-learn.org/sklearn-tutorial/auto_examples/plot_roc_crossval.html

import numpy as np
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler,SMOTE
from sklearn.metrics import auc
from sklearn.metrics import plot_roc_curve
from sklearn.model_selection import StratifiedKFold

# #############################################################################
# Run classifier with cross-validation and plot ROC curves

cv = StratifiedKFold(n_splits=5, random_state=12, shuffle=True)
plt.rcParams["figure.figsize"] = [14,10]
tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)
fig, ax = plt.subplots()
for i, (train_fold_index, val_fold_index) in enumerate(cv.split(X, Y)):
  X_train_fold,y_train_fold = X[train_fold_index], Y[train_fold_index]
  X_val_fold, y_val_fold = X[val_fold_index], Y[val_fold_index]
  smoter = SMOTE(random_state=12)
  X_train_fold_resample, y_train_fold_resample = smoter.fit_resample(X_train_fold,y_train_fold)
  classifier =XGBClassifier(n_estimators=9, max_depth=5, scale_pos_weight=2.47)

  classifier.fit(X_train_fold_resample,y_train_fold_resample)
  viz = plot_roc_curve(classifier, X[val_fold_index], Y[val_fold_index],
                         name='ROC fold {}'.format(i+1),
                         alpha=0.3, lw=1, ax=ax)
  interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
  interp_tpr[0] = 0.0
  tprs.append(interp_tpr)
  aucs.append(viz.roc_auc)

ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
        label='Chance', alpha=.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color='b',
        label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
        lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                label=r'$\pm$ 1 std. dev.')

ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05],
       title="Receiver operating characteristic")
ax.legend(loc="lower right")

plt.savefig("roc.jpg", dpi=300)

plt.show()


In [ ]:
n_classes = np.unique(Y).size

cv = StratifiedKFold(n_splits=5, random_state=12, shuffle=True)
xgb =  XGBClassifier(n_estimators=32, max_depth=3, scale_pos_weight=.2875)
metric=make_scorer(geometric_mean_score)

model = Pipeline([
        ('sampling', SMOTE(random_state=12)),
        ('classification', xgb)
    ])

score, permutation_scores, pvalue = permutation_test_score(
    model,X, Y, scoring=metric, cv=cv, n_permutations=1000)

print("Classification score %s (pvalue : %s)" % (score, pvalue))

## ROC Curve for 10-Fold Cross Validation with SMOTE oversampling 
 # Source: https://ogrisel.github.io/scikit-learn.org/sklearn-tutorial/auto_examples/plot_roc_crossval.html

import numpy as np
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler,SMOTE
from sklearn.metrics import auc
from sklearn.metrics import plot_roc_curve
from sklearn.model_selection import StratifiedKFold

# #############################################################################
# Run classifier with cross-validation and plot ROC curves

cv = StratifiedKFold(n_splits=5, random_state=12, shuffle=True)
plt.rcParams["figure.figsize"] = [14,10]
tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)
fig, ax = plt.subplots()
for i, (train_fold_index, val_fold_index) in enumerate(cv.split(X, Y)):
  X_train_fold,y_train_fold = X[train_fold_index], Y[train_fold_index]
  X_val_fold, y_val_fold = X[val_fold_index], Y[val_fold_index]
  smoter = SMOTE(random_state=12)
  X_train_fold_resample, y_train_fold_resample = smoter.fit_resample(X_train_fold,y_train_fold)
  classifier =XGBClassifier(n_estimators=32, max_depth=3, scale_pos_weight=.2875)
  classifier.fit(X_train_fold_resample,y_train_fold_resample)
  viz = plot_roc_curve(classifier, X[val_fold_index], Y[val_fold_index],
                         name='ROC fold {}'.format(i+1),
                         alpha=0.3, lw=1, ax=ax)
  interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
  interp_tpr[0] = 0.0
  tprs.append(interp_tpr)
  aucs.append(viz.roc_auc)

ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
        label='Chance', alpha=.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color='b',
        label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
        lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                label=r'$\pm$ 1 std. dev.')

ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05],
       title="Receiver operating characteristic")
ax.legend(loc="lower right")

plt.show()






# Permutation Testing


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import permutation_test_score
from imblearn.metrics import geometric_mean_score
from xgboost import XGBClassifier
from sklearn.metrics import make_scorer

#Uses test 1 described here:
# http://www.jmlr.org/papers/volume11/ojala10a/ojala10a.pdf

# #############################################################################
n_classes = np.unique(Y).size

cv = StratifiedKFold(n_splits=5, random_state=12, shuffle=True)
xgb =  XGBClassifier(n_estimators=9, max_depth=5, scale_pos_weight=2.47)
metric=make_scorer(geometric_mean_score)

score, permutation_scores, pvalue = permutation_test_score(
    xgb,X, Y, scoring=metric, cv=cv, n_permutations=1000)

print("Classification score %s (pvalue : %s)" % (score, pvalue))

# #############################################################################
# View histogram of permutation scores
plt.figure(figsize=(12,6))
plt.hist(permutation_scores, 20, label='Permutation scores',
         edgecolor='black')
ylim = plt.ylim()

plt.plot(2 * [score], ylim, '--g', linewidth=3,
         label='Classification Score')
plt.plot(2 * [1. /n_classes], ylim, '--k', linewidth=3, label='Luck')
plt.ylim(ylim)
plt.legend()
plt.xlabel('Score')
plt.savefig("ptest.jpg", dpi=300)

plt.show()



# Feature Selection

In [ ]:
# most improtant feature function
def Important_fetures(mymodel,featuredict):
    import numpy as np
    import sklearn as sk
    import sklearn.datasets as skd
    import matplotlib.pyplot as plt
    %matplotlib inline

    importances = model.feature_importances_

    indice = np.argsort(importances)[::-1]

    indices = indice [:30]

    # Print the feature ranking
    num=0
    with open("/content/drive/My Drive/CT Analysis/Saved Data/Important Features/XGBoost2.txt", "w") as txt_file:
      for f in indices:
        indexname = f;
        num+=1;
        if feature_cols[indexname] in featuredict:
          featuredict[feature_cols[indexname]][0] += 1
          featuredict[feature_cols[indexname]][1] += importances[indexname]
        else:
          featuredict[feature_cols[indexname]] = [1,importances[indexname]]


In [ ]:
featuredict = {}

for x in range(0,1000):

        cv = StratifiedKFold(n_splits=5, shuffle=True)

        Precisons = []
        Recalls = []
        F1s = []
        G_means = []

        for train_fold_index, val_fold_index in cv.split(X,Y):
          X_train_fold,y_train_fold = X[train_fold_index], Y[train_fold_index]
          X_val_fold, y_val_fold = X[val_fold_index], Y[val_fold_index]
          smoter = SMOTE()
          X_train_fold_resample, y_train_fold_resample = smoter.fit_resample(X_train_fold,y_train_fold)
          model = XGBClassifier(n_estimators=9, max_depth=5, scale_pos_weight=2.47)
          model.fit(X_train_fold_resample,y_train_fold_resample)
          pt = model.predict(X_val_fold)

          Important_fetures(model,featuredict)

          Precisons.append(precision_score(y_val_fold,pt))
          Recalls.append(recall_score(y_val_fold,pt))
          F1s.append(f1_score(y_val_fold,pt))
          G_means.append(geometric_mean_score(y_val_fold,pt))

In [ ]:
#List ranked by average
import operator
import collections

Avg = {}
Ocurr = {}
Tavg ={}
for key in featuredict:
  Avg[key] = [featuredict[key][1]/featuredict[key][0],featuredict[key][0]]
  Ocurr[key] = featuredict[key][0]
  Tavg[key] =  featuredict[key][1]/5000

AvgRank = sorted(Avg.items(),key=lambda kv: kv[1][0],reverse=True)
OcurrRank = sorted(Ocurr.items(),key=lambda x: x[1],reverse=True)
TavRank = sorted(Tavg.items(),key=lambda kv: kv[1],reverse=True)

sortedAvg = {}
for i in AvgRank:
  sortedAvg[i[0]] = [i[1][0],i[1][1]]



Ocuurpd = pd.DataFrame.from_dict(OcurrRank)
Avgdf = pd.DataFrame.from_dict(sortedAvg,orient='index',columns=['Avg.Value','Occurance'])
Tavdf = pd.DataFrame.from_dict(TavRank)
Ocuurpd.columns = ['Feature', 'Avg. Value']
Avgdf.to_csv('Average featuer Importance-hgd.CSV');
Ocuurpd.to_csv('Ocuurance-hgd.CSV')
Tavdf.to_csv('TSC-hgd.CSV')



In [ ]:
a = 0 
df2 = df
for (columnName, columnData) in df.iteritems():
  if columnName not in Avg:
    a +=1
    df2 = df2.drop(columnName, axis=1)

In [ ]:
#Creating labels

x1 = df2
Y = df["hgd_malignancy"].copy();
feature_cols = x1.columns

#Scale values from 0 to 1
scaler = MinMaxScaler(feature_range=(0, 1))
X1 = scaler.fit_transform(x1)

In [ ]:
cv = StratifiedKFold(n_splits=5, random_state=12, shuffle=True)

Precisons = []
Recalls = []
F1s = []
G_means = []
accuracy = []


for train_fold_index, val_fold_index in cv.split(X1,Y):
  X_train_fold,y_train_fold = X1[train_fold_index], Y[train_fold_index]
  X_val_fold, y_val_fold = X1[val_fold_index], Y[val_fold_index]
  smoter = SMOTE(random_state=12)
  X_train_fold_resample, y_train_fold_resample = smoter.fit_resample(X_train_fold,y_train_fold)
  model = XGBClassifier(n_estimators=9, max_depth=5, scale_pos_weight=2.47)
  model.fit(X_train_fold_resample,y_train_fold_resample)
  pt = model.predict(X_val_fold)

  print("confusion_matrix:")
  print(confusion_matrix(y_val_fold,pt))
  Precisons.append(precision_score(y_val_fold,pt))
  Recalls.append(recall_score(y_val_fold,pt))
  F1s.append(f1_score(y_val_fold,pt))
  G_means.append(geometric_mean_score(y_val_fold,pt))
  accuracy.append(accuracy_score(y_val_fold,pt))

print('Precision: ',mean(Precisons))
print('Recall: ',mean(Recalls))
print('F1: ',mean(F1s))
print('G_mean: ',mean(G_means))
print('Accuracy: ',mean(accuracy))
print(AvgRank)